In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!unzip "/content/drive/MyDrive/cubicasa5k.zip" -d "/content/cubicasa_dataset"

Streaming output truncated to the last 5000 lines.
   creating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/4014/
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/4014/F1_scaled.png  
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/4014/F1_original.png  
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/4014/model.svg  
   creating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14296/
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14296/F1_scaled.png  
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14296/F1_original.png  
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14296/model.svg  
   creating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14295/
  inflating: /content/cubicasa_dataset/cubicasa5k/high_quality_architectural/14295/F2_scaled.png  
  inflating:

In [26]:
pip install opencv-python matplotlib lxml tqdm pycocotools


In [27]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [28]:
!pip install torchvision --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [29]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-hbmgezm4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-hbmgezm4
  Resolved https://github.com/facebookresearch/detectron2.git to commit a1ce2f956a1d2212ad672e3c47d53405c2fe4312
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=6352513 sha256=6d19d782d7fbcf7875d7ca16aec0281bc9666ffa48b07fc49518995c86b25131
  Stored in directory: /tmp/pip-ephem-wheel-cache-4gn2twhd/wheels/d3/6e/bd/1969578f1456a6be2

In [57]:
!pip install lxml

In [52]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

def extract_svg_from_html(html_path):
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        svg_tag = soup.find('svg')
        if svg_tag:
            return ET.fromstring(str(svg_tag))
    return None

In [55]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

def extract_svg_root(svg_path):
    with open(svg_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, features="xml")  # Use XML parser
        svg_tag = soup.find('svg')
        if svg_tag:
            return ET.fromstring(str(svg_tag))
    return None

In [53]:
def parse_svg_root(svg_root):
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    rooms = []

    for g in svg_root.findall('.//svg:g', ns):
        label = g.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label', '')
        if label.startswith('room-') or label in ['Kitchen', 'Living Room', 'Bedroom']:  # fallback
            room_name = label.replace('room-', '')
            bbox = extract_bbox_from_group(g, ns)
            if bbox:
                rooms.append({'name': room_name, 'bbox': bbox})
    return rooms

In [56]:
svg_root = extract_svg_from_html("/content/cubicasa_dataset/cubicasa5k/colorful/10052/model.svg")
rooms = parse_svg_root(svg_root)
print("Rooms found:", rooms)

Rooms found: []


/tmp/ipython-input-3472145361.py:6: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(f, 'html.parser')
